In [1]:
%load_ext autoreload
%autoreload 2

import torch
from blora_utils import load_loras, prepare_batch
from transformers import LlamaForCausalLM, LlamaTokenizer
from IPython.display import clear_output

# from peft.tuners.lora import Linear
# from blora_utils import forward
# Linear.forward = forward

torch.set_default_tensor_type(torch.cuda.HalfTensor)

/home/ubuntu/minimal-llama/tenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "decapoda-research/llama-7b-hf"
model = LlamaForCausalLM.from_pretrained(model_path)
tokenizer = LlamaTokenizer.from_pretrained(model_path)
tokenizer.pad_token = 0

Loading checkpoint shards: 100%|██████████| 33/33 [00:10<00:00,  3.07it/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
loras = ["jondurbin/airoboros-7b-gpt4-1.2-peft", "trl-lib/llama-7b-se-rl-peft", "winddude/wizardLM-LlaMA-LoRA-7B"]
model, lora_map = load_loras(model, loras)

In [4]:
inputs = [('Outline a five sentence short story where a character stumbles upon a secret room in their house that contains relics from their future.',
  'jondurbin/airoboros-7b-gpt4-1.2-peft'),
 ('Write a 6 line dialogue between a character and a magical creature that only they can see.',
  'trl-lib/llama-7b-se-rl-peft'),
 ('Describe a four sentence scene where a character discovers a hidden talent that changes their life forever.',
  'winddude/wizardLM-LlaMA-LoRA-7B'),
 ('Sculpt a three verse poem about the feeling of walking through a lush, vibrant garden in full bloom.',
  'trl-lib/llama-7b-se-rl-peft'),
 ('Develop an eight sentence short story about a character who can bring their dreams into reality, but only for a limited time.',
  'winddude/wizardLM-LlaMA-LoRA-7B')]

batch_lora_ids = [inp[1] for inp in inputs]

batch = prepare_batch(inputs, tokenizer, model, lora_map)

In [5]:
outputs = []

for out in model.generate(
    **batch,
    max_length=200,
    stream_output=True
):
    outputs.append(out)
    batch_decoded = tokenizer.batch_decode(torch.cat([out.reshape(-1, 1) for out in outputs], dim=1))
    clear_output(wait=True)
    print("\n\n".join([lora + ":\n" + prompt + '\n' + decoded for (prompt, lora), decoded in zip(inputs, batch_decoded)]))

jondurbin/airoboros-7b-gpt4-1.2-peft:
Outline a five sentence short story where a character stumbles upon a secret room in their house that contains relics from their future.

The character, who is a young boy named Timmy, stumbles upon a secret room in his house that contained relics from his future. The room was hidden behind a bookcase in the library, and it was filled with strange artifacts and documents.
Timmy's curiosity got the best of him, and he decided to explore the room. As he opened the door, he was greeted by a strange man wearing a long coat and a top hat. The man introduced himself as Dr. Time, and he explained that he was from the future.
Dr. Time told Timmy that he had traveled back in time to prevent a catastrophic event from happening. He had discovered a way to manipulate time, but it required the use of these relics.
Timmy was

trl-lib/llama-7b-se-rl-peft:
Write a 6 line dialogue between a character and a magical creature that only they can see.

Write a 6 line di